In [34]:
import numpy as np
import csv
import pandas as pd
import random
from scipy.sparse import *
from sklearn.utils import shuffle 
import time

In [35]:
NUM=18771  #NUM为节点个数

In [36]:
data = pd.read_table('ca-AstroPh.txt', delimiter=' ')
data.columns = ['vertex','neighbour']

In [37]:
data

,vertex,neighbour
0,1,2
1,1,3
2,1,4
3,1,5
4,1,6
...,...,...
198045,18765,18766
198046,18765,18767
198047,18766,18767
198048,18768,18769


In [38]:
original_matrix = np.zeros((18771, 18771)) #注意第0列对应vertex 1

In [39]:
for i in range(198050):
    original_matrix[data.iloc[i]['vertex']-1][data.iloc[i]['neighbour']-1]+=1
    original_matrix[data.iloc[i]['neighbour']-1][data.iloc[i]['vertex']-1]+=1

In [40]:
original_matrix

array([[0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]])

original_matrix:OM是对称的,我们把它的一列看作一个对象

如果关心的是高效修改 - 使用 DOK、LIL 或 COO。这些通常用于构建矩阵。
如果关心的是有效的访问和矩阵操作 - 使用 CSR 或 CSC

In [41]:

csc= csc_matrix(original_matrix)
index_pointer=csc.indptr
indices=csc.indices

In [42]:

def genRandomPermutationMatrix(index, numOfhash):  #generate a random
    random_permutation_matrix = index
    index_to_shuffle = random_permutation_matrix.copy()
    new_index = [shuffle(index_to_shuffle, random_state=i + np.random.randint(low=1, high=2**30)) for i in range(numOfhash)]
    random_permutation_matrix = np.c_[new_index].T
    return random_permutation_matrix
#numOfhash=num of the times of permutation,cause a hashfunction correspond to a permutation way
def genSigMatrix(csc, numOfhash, index_pointer, indices):
    index = np.arange(csc.shape[0]) #arange return a ndarray
    random_permutation_matrix = genRandomPermutationMatrix(index, numOfhash) 
    col_num = csc.shape[1]
    random_col = random_permutation_matrix.shape[1] #randomly pick a col
    sig_matrix = np.zeros((random_col, col_num))

    for j in range(random_col):
        for i in range(col_num):
            sig_matrix[j][i] = min(random_permutation_matrix[:, j][indices[index_pointer[i]:index_pointer[i+1]]])

    return sig_matrix


In [43]:
import hashlib


def LSH(sig_matrix, r, b):
    rownum = sig_matrix.shape[0]
    colnum = sig_matrix.shape[1]
    begin , end = 0, r
    count = 0

    index = {}
    buckets={}
    while end <= rownum:
        count += 1

        for col in range(colnum):
            hash_obj = hashlib.md5()
            band = sig_matrix[begin: end, col].tobytes() + np.array([count]).tobytes()
            hash_obj.update(band)
            tag = hash_obj.hexdigest()
            if col not in index:
                index[col] = set()
                index[col].add(tag)
            else:
                index[col].add(tag)

            if tag in buckets:
                buckets[tag].add(col)
            else:
                buckets[tag] = set()
                buckets[tag].add(col)
        begin += r
        end += r

    return buckets, index

In [44]:
def jaccard(csc,col1,col2):
    indptr = csc.indptr
    indices = csc.indices
    a=set()
    b=set()
    for i in indices[indptr[col1]:indptr[col1+1]]:
        a.add(i)
    for j in indices[indptr[col2]:indptr[col2+1]]:
        b.add(j)
    return len(set(a).intersection(set(b))) / len(set(a).union(set(b)))

In [45]:
from collections import Counter #用于构造一个拿来排序的集合
def searchsimilar(csc,index,hashBucks,query):
    result=set()  #返回跟query在一个桶里的对象(用set保证了不重复)
    for tag in index[query]:
        if query in hashBucks[tag]:
            for i in hashBucks[tag]:
                result.add(i)
    result.remove(query) #去除查询对象自己
    
    jaccard_dict={}
    for i in result:
        jaccard_dict[i]=jaccard(csc,query,i)
    sorted_list=sorted(jaccard_dict.items(),key=lambda x:-x[1])
    result.clear()
    count=0
    while(count<len(sorted_list)):
        if(count==10):
            break
        result.add(sorted_list[count][0])
        count+=1
    return result

In [129]:
b,r=40,3
n=b*r
time_start=time.time()
sig_matrix=genSigMatrix(csc,n,index_pointer,indices)
time_end=time.time() 
time_c= time_end - time_start   #运行所花时间
print("genSigMatrix用时：",end='')
print(time_c)

genSigMatrix用时：8.017287254333496


In [140]:
time_start=time.time()
hashBucks,index=LSH(sig_matrix,r,b)
time_end=time.time() 
time_c= time_end - time_start   #运行所花时间
print("genhashBuckets用时：",end='')
print(time_c)

genhashBuckets用时：1.1480531692504883


In [48]:
# #举个例子，用querycol=132来查询
# querycol=132
# similarset=searchsimilar(csc,index,hashBucks,querycol)

In [104]:
def precision(csc,querycol,similarset,k):
    vertexdict={}
    for i in range(NUM):
        if(i==querycol):
            continue
        vertexdict[i]=jaccard(csc,querycol,i)
    sorted_list=sorted(vertexdict.items(),key=lambda x:-x[1])
    top10=set()
    topk=set()
    count=1
    for item in sorted_list:
        top10.add(item[0])
        if count<=k:
            topk.add(item[0])
        if count==10:
            break
        count+=1
    result=len(top10.intersection(similarset)) / 10
    if k==0:
        result2=0
    else:
        result2=len(topk.intersection(similarset))/k
    return result,result2

In [50]:
# result=precision(csc,querycol,similarset)
# print(similarset)
# print(result)

In [142]:
#用系统抽样的方法抽取一些节点来求searchsimilar的平均用时，与平均precision

#precision_top10=len(similarset & real_top10_set)/10
#当找到的候选集similarset的大小k小于10时，
#precision_topk=len(similarset & real_topk_set)/k # k=len(similarset)

precision_top10,precision_topk=0,0
query_list=np.arange(0,NUM,200)
print("查询节点个数：",end='')
print(len(query_list))
sum_search_time=0
for query in query_list:
    time_start=time.time()
    similarset=searchsimilar(csc,index,hashBucks,query)
    time_end=time.time()
    sum_search_time+=time_end-time_start
    k=len(similarset)
    top10,topk=precision(csc,query,similarset,k)
    precision_top10+=top10
    precision_topk+=topk
print("average search time:",end='')
print(sum_search_time/len(query_list))
print("precison_top10:",end=' ')
print(precision_top10/len(query_list))
print("precision_topk:",end=' ')
print(precision_topk/len(query_list))

查询节点个数：94
average search time:0.0
precison_top10: 0.15957446808510634
precision_topk: 0.31959219858156024
